<a href="https://colab.research.google.com/github/deviwlndr/714220054_Devi-Wulandari/blob/main/analisis_sentimen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install Sastrawi
!pip install googletrans==4.0.0-rc1 langdetect



In [7]:
import pandas as pd
import numpy as np
import re
import string
import nltk

from langdetect import detect
from googletrans import Translator

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from wordcloud import WordCloud
import matplotlib.pyplot as plt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [8]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [9]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Analisis Sentimen/dataset_ulasan_museum_sri_baduga.csv')
df.head()


,title,url,stars,name,reviewUrl,text
0,Museum Sri Baduga,https://www.google.com/maps/search/?api=1&quer...,4,NaN,NaN,NaN
1,Museum Sri Baduga,https://www.google.com/maps/search/?api=1&quer...,5,NaN,NaN,Musieum Sribaduga
2,Museum Sri Baduga,https://www.google.com/maps/search/?api=1&quer...,5,NaN,NaN,NaN
3,Museum Sri Baduga,https://www.google.com/maps/search/?api=1&quer...,5,NaN,NaN,Museum di dalam kota bandung
4,Museum Sri Baduga,https://www.google.com/maps/search/?api=1&quer...,4,NaN,NaN,هذا المتحف هو رحلة عبر الزمن، حيث نرى بعض من ت...


In [10]:
df = df[['text', 'stars']]
df = df.dropna(subset=['text'])
df['stars'] = df['stars'].astype(int)

df.head()


,text,stars
1,Musieum Sribaduga,5
3,Museum di dalam kota bandung,5
4,هذا المتحف هو رحلة عبر الزمن، حيث نرى بعض من ت...,4
7,"Kesini pas study tour SMP, mungkin tahun 2018/...",5
8,Bagus sekali,5


In [11]:
def case_folding(text):
    return text.lower()

In [12]:
df['case_folding'] = df['text'].apply(case_folding)

In [13]:
# 3. CHARACTER FILTERING
# Menghapus angka, emoji, tanda baca, URL
# ---------------------------------------------
def clean_characters(text):
    text = re.sub(r'http\S+|www\S+|https\S+', ' ', text)   # hapus URL
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)              # hapus angka, emoji, simbol
    text = re.sub(r'\s+', ' ', text).strip()              # rapikan spasi
    return text

In [14]:
df['clean_text'] = df['case_folding'].apply(clean_characters)

In [15]:
translator = Translator()

def translate_to_indonesian(text):
    if not isinstance(text, str) or text.strip() == "":
        return text

    try:
        lang = detect(text)
    except:
        return text

    if lang == "en":  # hanya terjemahkan jika English
        try:
            return translator.translate(text, src='en', dest='id').text
        except:
            return text

    return text



In [16]:
df['translated'] = df['clean_text'].apply(translate_to_indonesian)
print(df[['clean_text','translated']].head())


                                clean_text  \
1                        musieum sribaduga   
3             museum di dalam kota bandung   
4                                            
7  kesini pas study tour smp mungkin tahun   
8                             bagus sekali   

                                translated  
1                        musieum sribaduga  
3             museum di dalam kota bandung  
4                                           
7  kesini pas study tour smp mungkin tahun  
8                             bagus sekali  


In [17]:
# 4. TOKENIZING

nltk.download('punkt')

def tokenizing(text):
    return nltk.word_tokenize(text)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
df['tokenized'] = df['translated'].apply(tokenizing)

In [19]:
# Kamus normalisasi (bisa ditambah)
normalization_dict = {
    # --- Kata Informal / Slang ---
    "ga": "tidak",
    "gk": "tidak",
    "gak": "tidak",
    "tdk": "tidak",
    "engga": "tidak",
    "enggak": "tidak",
    "nggak": "tidak",
    "ngga": "tidak",

    "bgt": "banget",
    "bngt": "banget",
    "bangettt": "banget",

    "cmn": "cuma",
    "cman": "cuma",
    "cuma2": "cuma",

    "trs": "terus",
    "trus": "terus",

    "tmn": "teman",
    "tmn2": "teman",

    "udh": "sudah",
    "udah": "sudah",
    "udhh": "sudah",

    "blm": "belum",
    "belom": "belum",

    "syg": "sayang",

    "krg": "kurang",

    "sblm": "sebelum",

    "dr": "dari",
    "drpd": "daripada",

    "klo": "kalau",
    "kalo": "kalau",

    "pls": "tolong",
    "plis": "tolong",

    "aja": "saja",
    "aj": "saja",

    "ni": "ini",
    "nih": "ini",

    "tu": "itu",
    "ituu": "itu",

    "bru": "baru",
    "baru2": "baru",

    "lok": "lokasi",

    "pas": "saat",

    # --- Singkatan Lokasi / Umum ---
    "bndng": "bandung",
    "bdg": "bandung",
    "jbr": "jawa barat",

    # --- Typos Umum ---
    "bgs": "bagus",
    "baguss": "bagus",
    "bgus": "bagus",

    "bnyk": "banyak",
    "bnyknya": "banyaknya",

    "tmpt": "tempat",
    "tempet": "tempat",

    "rame": "ramai",
    "ramee": "ramai",

    "nyamn": "nyaman",
    "nyamann": "nyaman",

    "lumayanlah": "lumayan",

    "mantappp": "mantap",
    "mantab": "mantap",
    "mantul": "mantap",

    # --- Istilah Museum / Edukasi ---
    "edukasii": "edukasi",
    "edukatif": "edukatif",
    "koleksinyaa": "koleksinya",
    "sejarahh": "sejarah",

    # --- Bahasa Inggris (umum di Google Maps) ---
    "good": "bagus",
    "nice": "bagus",
    "great": "bagus",
    "awesome": "bagus",
    "amazing": "bagus",
    "cool": "keren",
    "clean": "bersih",
    "recommended": "direkomendasikan",
    "recommend": "rekomendasi",
    "worth": "layak",
    "worthit": "layak",
    "worthwhile": "layak",
    "cheap": "murah",
    "expensive": "mahal",
    "great": "bagus",
    "nice": "bagus",

    # --- Bahasa Sunda (umum dalam ulasan Museum Sri Baduga) ---
    "teh": "",         # partikel (dapat dihapus)
    "mah": "",         # partikel penegas
    "atuh": "",        # ekspresi
    "pisan": "sekali",
    "teteh": "kakak",
    "kenging": "boleh",
    "alus": "bagus",
    "gawe": "kerja",

    # --- Variasi ejaan tidak baku ---
    "seblm": "sebelum",
    "seblumnya": "sebelumnya",
    "baguss": "bagus",
    "bersiih": "bersih",
    "bersiiih": "bersih",
    "bersihh": "bersih",

    "murahh": "murah",
    "murahhh": "murah",

    "lengkappp": "lengkap",

    "worthittt": "layak",

    # --- Kata sambung / filler ---
    "the": "the",          # tetap (kadang judul)
    "of": "of",            # tetap
    "ya": "",              # filler
    "yah": "",             # filler
    "loh": "",             # filler
    "kok": "",             # filler
    "kan": "",             # filler
    "lah": "",             # filler

    # --- Variasi Pengulangan Huruf (contoh umum) ---
    "bagusss": "bagus",
    "kerenn": "keren",
    "keren banget": "keren",
    "kenyanggg": "kenyang",
    "luasss": "luas",
    "seruu": "seru",
    "seruuu": "seru",
    "harganyaa": "harganya",

    # --- Kata Google Maps yang sering muncul ---
    "service": "layanan",
    "staff": "staf",
    "facility": "fasilitas",
    "facilities": "fasilitas",
    "parking": "parkir",
    "parkiran": "parkir",
    "recommended": "direkomendasikan",
    "experience": "pengalaman",
    "view": "pemandangan",
    "history": "sejarah",
    "historical": "sejarah",

    # --- Tambahan umum ---
    "yg": "yang",
    "aja": "saja",
    "gini": "begini",
    "gitu": "begitu",
    "sini": "sini",
    "situ": "situ",
    "kemaren": "kemarin",
    "kmrn": "kemarin",
    "kmrn2": "kemarin",
    "besok2": "besok",
    "jd": "jadi",
    "jg": "juga",
    "tll": "terlalu",
    "sgt": "sangat"
}

def normalize_text(tokens):
    return [normalization_dict.get(word, word) for word in tokens]


In [20]:
df['normalized'] = df['tokenized'].apply(normalize_text)

In [21]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

factory = StopWordRemoverFactory()
stopwords_id = set(factory.get_stop_words())

additional_stopwords = {
    "banget", "sekali", "kali",
    "ya", "yah", "iya",
    "sih", "nih", "tuh",
    "kayak", "kayaknya",
    "dong", "deh", "kok",
    "gitu", "gini",
    "aja", "doang",
    "bikin", "biar", "buat", "jadi",
    "tempat", "lokasi", "area",
    "museum", "to", "we", "about"
}

stopwords_final = stopwords_id.union(additional_stopwords)

def remove_stopwords(tokens):
    return [word for word in tokens if word not in stopwords_final]



In [22]:
df['stopword_removed'] = df['normalized'].apply(remove_stopwords)

In [23]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem_sentence(tokens):
    sentence = " ".join(tokens)         # gabungkan list → string
    stemmed = stemmer.stem(sentence)    # stemming kalimat
    return stemmed.split()              # pecah lagi → list




In [24]:
df["stemmed_text"] = df["stopword_removed"].apply(stem_sentence)
df['stemmed_joined'] = df['stemmed_text'].apply(lambda x: " ".join(x))

In [25]:
import pandas as pd

# URL resmi InSet Lexicon
url_positive = "https://raw.githubusercontent.com/fajri91/InSet/master/positive.tsv"
url_negative = "https://raw.githubusercontent.com/fajri91/InSet/master/negative.tsv"

# Membaca lexicon dari GitHub tanpa header bawaan file
lex_pos = pd.read_csv(url_positive, sep="\t", header=None, names=["word", "weight"], skiprows=1)
lex_neg = pd.read_csv(url_negative, sep="\t", header=None, names=["word", "weight"], skiprows=1)

# Gabungkan
lexicon_df = pd.concat([lex_pos, lex_neg], ignore_index=True)

# Konversi weight ke integer (AMAN sekarang)
lexicon_df["weight"] = lexicon_df["weight"].astype(int)

# Jadikan dictionary
lexicon = dict(zip(lexicon_df["word"], lexicon_df["weight"]))

print("Jumlah kata dalam lexicon:", len(lexicon))


Jumlah kata dalam lexicon: 9074


In [26]:
# Ambil bobot maksimum untuk tiap kata → lebih logis untuk sentimen
lexicon_clean = lexicon_df.groupby("word", as_index=False)["weight"].max()

# Ubah ke dictionary
lexicon_final = dict(zip(lexicon_clean["word"], lexicon_clean["weight"]))

print("Jumlah kata di lexicon_final:", len(lexicon_final))


Jumlah kata di lexicon_final: 9074


In [34]:
def get_lexicon_score(tokens, lexicon):
    if not isinstance(tokens, list):
        return 0
    return sum(lexicon.get(word, 0) for word in tokens)

df["lexicon_score"] = df["stemmed_text"].apply(lambda x: get_lexicon_score(x, lexicon_final))


In [37]:
def label_sentiment(score):
    if score > 0:
        return "positif"
    elif score < 0:
        return "negatif"
    else:
        return "netral"

df["sentiment"] = df["lexicon_score"].apply(label_sentiment)


In [38]:
df[["stemmed_text", "lexicon_score", "sentiment"]].head(20)


,stemmed_text,lexicon_score,sentiment
1,"[musieum, sribaduga]",0,netral
3,"[kota, bandung]",-2,negatif
4,[],0,netral
7,"[kesini, study, tour, smp, mungkin, tahun]",-1,negatif
8,[bagus],2,positif
9,[mantap],5,positif
10,"[seru, anak, tau, sejarah, banyak, ajar]",2,positif
16,[mantap],5,positif
17,"[sangat, baik, edukasi, anak, didik, koleksi, ...",9,positif
18,"[datang, sini, waktu, pamer, nasional, pusaka,...",3,positif


In [39]:
df['sentiment'].value_counts().reset_index().rename(columns={'index': 'sentiment', 'sentiment': 'jumlah'})


,jumlah,count
0,positif,1044
1,negatif,228
2,netral,220
